앞 장에서는 단일 데이터셋의 집계 방법을 알아보았음<br/>
하지만 스파크 애플리케이션에서는 다양한 데이터셋을 함께 결합해 사용하는 경우가 더 많음<br/>
따라서 조인은 거의 모든 스파크 작업에 필수적으로 사용됨<br/>
스파크는 서로 다른 데이터를 조합할 수 있으므로 데이터를 처리할 때 기업의 여러 데이터소스를 활용할 수 있음<br/>

이 장에서는 스파크가 지원하는 조인 타입과 사용법 그리고 실제 스파크가 클러스터에서 어떻게 조인을 실행하는지 생각해볼 수 있도록 기본적인 내부 동작 방식을 다룸<br/>
이러한 기초 지식은 *메모리 부족 상황을 회피하는 방법*과 이전에 풀지 못했던 문제를 해결하는 데 도움이 됨<br/>

# 8.1 조인 표현식

스파크는 **왼쪽**과 **오른쪽** 데이터셋에 있는 하나 이상의 **키값**을 비교하고 왼쪽 데이터셋과 오른쪽 데이터셋의 결합 여부를 결정하는 조인 표현식(join expression)의 평가 결과에 따라 두 개의 데이터셋을 조인함<br/>
가장 많이 사용하는 조인 표현식은 왼쪽과 오른쪽 데이터셋에 지정된 키가 동일한지 비교하는 동등 조인(equal-join)임<br/>
키가 일치하면 스파크는 왼쪽과 오른쪽의 데이터셋을 결합함<br/>
일치하지 않으면 데이터셋을 결합하지 않음<br/>
스파크는 일치하는 키가 없는 로우는 조인에 포함시키지 않음<br/>
스파크는 동등 조인뿐만 아니라 더 복잡한 조인 정책도 지원함<br/>
또한 복합 데이터 타입을 조인에 사용할 수도 있음<br/>
예를 들어 배열 타입의 키에 조인할 키가 존재하는지 확인해 조인을 수행할 수 있음<br/>

# 8.2 조인 타입

조인 표현식은 두 로우의 조인 여부를 결정하는 반면 조인 타입은 결과 데이터셋에 어떤 데이터가 있어야 하는지 결정함<br/>
스파크에서 사용할 수 있는 조인 타입은 다음과 같음<br/>
* 내부 조인(inner join): 왼쪽과 오른쪽 데이터셋에 키가 있는 로우를 유지
* 외부 조인(outer join): 왼쪽이나 오른쪽 데이터셋에 키가 있는 로우를 유지
* 왼쪽 외부 조인(left outer join): 왼쪽 데이터셋에 키가 있는 로우를 유지
* 오른쪽 외부 조인(right outer join): 오른쪽 데이터셋에 키가 있는 로우를 유지
* 왼쪽 세미 조인(left semi join): 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우에는 키가 일치하는 왼쪽 데이터셋만 유지
* 왼쪽 안티 조인(left anti join): 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우에는 키가 일치하지 않는 왼쪽 데이터셋만 유지
* 자연 조인(natural join): 두 데이터셋에서 동일한 이름을 가진 컬럼을 암시적(implicit)으로 결합하는 조인을 수행
* 교차 조인(cross join) 또는 카테시안 조인(Cartesian join): 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터셋의 모든 로우를 조합

이전에 관계형 데이터베이스 시스템이나 엑셀 스프레드시트를 사용해봤다면 두 개의 데이터셋을 조인하는 것이 그리 어렵지 않을 것임<br/>
이제 각 조인 타입에 대한 예제를 살펴보겠음<br/>
그러면 문제를 해결하기 위해 정확하게 어떤 조인 타입을 사용해야 하는지 쉽게 이해할 수 있게 될 것임<br/>
우선 예제에서 사용할 몇 가지 간단한 데이터셋을 만듦<br/>

In [1]:
val person = Seq(
    (0, "Bill Chambers", 0, Seq(100)),
    (1, "Matei Zaharia", 1, Seq(500, 250, 100)),
    (2, "Michael Armbrust", 1, Seq(250, 100)))
    .toDF("id", "name", "graduate_program", "spark_status")

val graduateProgram = Seq(
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D", "EECS", "UC Berkeley"))
    .toDF("id", "degree", "department", "school")

val sparkStatus = Seq(
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor"))
    .toDF("id", "status")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4045
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1641459237248)
SparkSession available as 'spark'


person: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]
graduateProgram: org.apache.spark.sql.DataFrame = [id: int, degree: string ... 2 more fields]
sparkStatus: org.apache.spark.sql.DataFrame = [id: int, status: string]


생성한 데이터셋을 이 장 전체 예제에서 사용하기 위해 테이블로 등록함<br/>

In [2]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [3]:
spark.sql("SELECT * FROM person").show(false)

+---+----------------+----------------+---------------+
|id |name            |graduate_program|spark_status   |
+---+----------------+----------------+---------------+
|0  |Bill Chambers   |0               |[100]          |
|1  |Matei Zaharia   |1               |[500, 250, 100]|
|2  |Michael Armbrust|1               |[250, 100]     |
+---+----------------+----------------+---------------+



In [4]:
spark.sql("SELECT * FROM graduateProgram").show(false)

+---+-------+---------------------+-----------+
|id |degree |department           |school     |
+---+-------+---------------------+-----------+
|0  |Masters|School of Information|UC Berkeley|
|2  |Masters|EECS                 |UC Berkeley|
|1  |Ph.D   |EECS                 |UC Berkeley|
+---+-------+---------------------+-----------+



In [5]:
spark.sql("SELECT * FROM sparkStatus").show(false)

+---+--------------+
|id |status        |
+---+--------------+
|500|Vice President|
|250|PMC Member    |
|100|Contributor   |
+---+--------------+



# 8.3 내부 조인

내부 조인은 DataFrame이나 테이블에 존재하는 키를 평가함<br/>
그리고 true로 평가되는 로우만 결합함<br/>
다음은 *graduateProgram* DataFrame과 *person* DataFrame을 조인해 새로운 DataFrame을 만드는 예제임<br/>

In [6]:
val joinExpression = person.col("graduate_program") === graduateProgram.col("id")

joinExpression: org.apache.spark.sql.Column = (graduate_program = id)


두 DataFrame 모두에 키가 존재하지 않으면 결과 DataFrame에서 볼 수 없음<br/>
예를 들어 다음과 같은 표현식을 사용하면 비어 있는 결과 DataFrame을 얻게 됨<br/>

In [7]:
val wrongJoinExpression = person.col("name") === graduateProgram.col("school")

wrongJoinExpression: org.apache.spark.sql.Column = (name = school)


내부 조인은 기본 조인 방식이므로 JOIN 표현식에 왼쪽 DataFrame과 오른쪽 DataFrame을 지정하기만 하면 됨<br/>

In [8]:
person.join(graduateProgram, joinExpression).show(false)

+---+----------------+----------------+---------------+---+-------+---------------------+-----------+
|id |name            |graduate_program|spark_status   |id |degree |department           |school     |
+---+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0  |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|2  |Michael Armbrust|1               |[250, 100]     |1  |Ph.D   |EECS                 |UC Berkeley|
|1  |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D   |EECS                 |UC Berkeley|
+---+----------------+----------------+---------------+---+-------+---------------------+-----------+



join 메서드의 세 번째 파라미터(joinType)로 조인 타입을 명확하게 지정할 수도 있음<br/>

In [9]:
var joinType = "inner"

person.join(graduateProgram, joinExpression, joinType).show(false)

+---+----------------+----------------+---------------+---+-------+---------------------+-----------+
|id |name            |graduate_program|spark_status   |id |degree |department           |school     |
+---+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0  |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|2  |Michael Armbrust|1               |[250, 100]     |1  |Ph.D   |EECS                 |UC Berkeley|
|1  |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D   |EECS                 |UC Berkeley|
+---+----------------+----------------+---------------+---+-------+---------------------+-----------+



joinType: String = inner


# 8.4 외부 조인

외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가하여 true나 false로 평가한 로우를 포함(그리고 조인)함<br/>
왼쪽이나 오른쪽 DataFrame에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입함<br/>

In [10]:
joinType = "outer"

person.join(graduateProgram, joinExpression, joinType).show(false)

+----+----------------+----------------+---------------+---+-------+---------------------+-----------+
|id  |name            |graduate_program|spark_status   |id |degree |department           |school     |
+----+----------------+----------------+---------------+---+-------+---------------------+-----------+
|1   |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D   |EECS                 |UC Berkeley|
|2   |Michael Armbrust|1               |[250, 100]     |1  |Ph.D   |EECS                 |UC Berkeley|
|null|null            |null            |null           |2  |Masters|EECS                 |UC Berkeley|
|0   |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+---------------------+-----------+



joinType: String = outer


# 8.5 왼쪽 외부 조인

왼쪽 외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가함<br/>
그리고 왼쪽 DataFrame의 모든 로우와 왼쪽 DataFrame과 일치하는 오른쪽 DataFrame의 로우를 함께 포함함<br/>
오른쪽 DataFrame에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입함<br/>

In [12]:
joinType = "left_outer"

graduateProgram.join(person, joinExpression, joinType).show(false)

+---+-------+---------------------+-----------+----+----------------+----------------+---------------+
|id |degree |department           |school     |id  |name            |graduate_program|spark_status   |
+---+-------+---------------------+-----------+----+----------------+----------------+---------------+
|0  |Masters|School of Information|UC Berkeley|0   |Bill Chambers   |0               |[100]          |
|2  |Masters|EECS                 |UC Berkeley|null|null            |null            |null           |
|1  |Ph.D   |EECS                 |UC Berkeley|2   |Michael Armbrust|1               |[250, 100]     |
|1  |Ph.D   |EECS                 |UC Berkeley|1   |Matei Zaharia   |1               |[500, 250, 100]|
+---+-------+---------------------+-----------+----+----------------+----------------+---------------+



joinType: String = left_outer


# 8.6 오른쪽 외부 조인

오른쪽 외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가함<br/>
그리고 오른쪽 DataFrame의 모든 로우와 오른쪽 DataFrame과 일치하는 왼쪽 DataFrame의 로우를 함께 포함함<br/>
왼쪽 DataFrame에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입함<br/>

In [13]:
joinType = "right_outer"

person.join(graduateProgram, joinExpression, joinType).show(false)

+----+----------------+----------------+---------------+---+-------+---------------------+-----------+
|id  |name            |graduate_program|spark_status   |id |degree |department           |school     |
+----+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0   |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|null|null            |null            |null           |2  |Masters|EECS                 |UC Berkeley|
|2   |Michael Armbrust|1               |[250, 100]     |1  |Ph.D   |EECS                 |UC Berkeley|
|1   |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D   |EECS                 |UC Berkeley|
+----+----------------+----------------+---------------+---+-------+---------------------+-----------+



joinType: String = right_outer


# 8.7 왼쪽 세미 조인

세미 조인은 오른쪽 DataFrame의 어떤 값도 포함하지 않기 때문에 다른 조인 탕비과는 약간 다름<br/>
단지 두 번째 DataFrame은 값이 존재하는지 확인하기 위해 값만 비교하는 용도로 사용함<br/>
만약 값이 존재한다면 왼쪽 DataFrame에 중복 키가 존재하더라도 해당 로우는 결과에 포함됨<br/>
왼쪽 세미 조인은 기존 조인 기능과는 달리 DataFrame의 필터 정도로 볼 수 있음<br/>

In [14]:
joinType = "left_semi"

graduateProgram.join(person, joinExpression, joinType).show(false)

+---+-------+---------------------+-----------+
|id |degree |department           |school     |
+---+-------+---------------------+-----------+
|0  |Masters|School of Information|UC Berkeley|
|1  |Ph.D   |EECS                 |UC Berkeley|
+---+-------+---------------------+-----------+



joinType: String = left_semi


In [15]:
val gradProgram2 = graduateProgram.union(Seq(
    (0, "Masters", "Duplicated Row", "Duplicated School")).toDF())

gradProgram2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, degree: string ... 2 more fields]


In [16]:
gradProgram2.createOrReplaceTempView("gradProgram2")
gradProgram2.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  1|   Ph.D|                EECS|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



# 8.8 왼쪽 안티 조인

왼쪽 안티 조인은 왼쪽 세미 조인의 반대 개념임<br/>
왼쪽 세미 조인처럼 오른쪽 DataFrame의 어떤 값도 포함하지 않음<br/>
단지 두 번째 DataFrame은 값이 존재하는지 확인하기 위해 값만 비교하는 용도로 사용함<br/>
하지만 두 번째 DataFrame에 존재하는 값을 유지하는 대신 두 번째 DataFrame에서 관련된 키를 찾을 수 없는 로우만 결과에 포함함<br/>
안티 조인은 SQL의 NOT IN과 같은 스타일의 필터로 볼 수 있음<br/>

In [17]:
joinType = "left_anti"

graduateProgram.join(person, joinExpression, joinType).show(false)

+---+-------+----------+-----------+
|id |degree |department|school     |
+---+-------+----------+-----------+
|2  |Masters|EECS      |UC Berkeley|
+---+-------+----------+-----------+



joinType: String = left_anti


# 8.9 자연 조인

자연 조인은 조인하려는 컬럼을 암시적으로 추정함<br/>
즉, 일치하는 컬럼을 찾고 그 결과를 반환함<br/>
왼쪽과 오른쪽 그리고 외부 자연 조인을 사용할 수 있음<br/>

*CAUTION*<br/>
암시적인 처리는 언제나 위험함<br/>
아래 쿼리는 두 DataFrame 또는 테이블이 id라는 동일한 컬럼명을 가지지만 각각의 데이터셋 입장에서는 서로 다른 의미를 지니므로 부정확한 결과를 낳을 수 있음<br/>
그렇기 때문에 자연 조인은 언제나 조심해서 사용해야 함<br/>

In [ ]:
/*
-- SQL

SELECT * FROM graduateProgram NATURAL JOIN person
*/

# 8.10 교차 조인(카테시안 조인)

마지막으로 알아볼 조인 타입은 교차 조인(또는 카테시안 조인)임<br/>
간단히 말해, 교차 조인은 조건절을 기술하지 않은 내부 조인을 의미함<br/>
교차 조인은 왼쪽 DataFrame의 모든 로우를 오른쪽 DataFrame의 모든 로우와 결합함<br/>

In [23]:
joinType = "cross"

graduateProgram.join(person, joinExpression, joinType).show(false)

+---+-------+---------------------+-----------+---+----------------+----------------+---------------+
|id |degree |department           |school     |id |name            |graduate_program|spark_status   |
+---+-------+---------------------+-----------+---+----------------+----------------+---------------+
|0  |Masters|School of Information|UC Berkeley|0  |Bill Chambers   |0               |[100]          |
|1  |Ph.D   |EECS                 |UC Berkeley|2  |Michael Armbrust|1               |[250, 100]     |
|1  |Ph.D   |EECS                 |UC Berkeley|1  |Matei Zaharia   |1               |[500, 250, 100]|
+---+-------+---------------------+-----------+---+----------------+----------------+---------------+



joinType: String = cross


교차 조인이 필요한 경우 다음과 같이 명시적으로 메서드를 호출할 수도 있음

In [24]:
person.crossJoin(graduateProgram).show(false)

+---+----------------+----------------+---------------+---+-------+---------------------+-----------+
|id |name            |graduate_program|spark_status   |id |degree |department           |school     |
+---+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0  |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1  |Matei Zaharia   |1               |[500, 250, 100]|0  |Masters|School of Information|UC Berkeley|
|2  |Michael Armbrust|1               |[250, 100]     |0  |Masters|School of Information|UC Berkeley|
|0  |Bill Chambers   |0               |[100]          |2  |Masters|EECS                 |UC Berkeley|
|1  |Matei Zaharia   |1               |[500, 250, 100]|2  |Masters|EECS                 |UC Berkeley|
|2  |Michael Armbrust|1               |[250, 100]     |2  |Masters|EECS                 |UC Berkeley|
|0  |Bill Chambers   |0               |[100]          |1  |Ph.D   |EECS           

# 8.11 조인 사용 시 문제점

조인을 하다 보면 몇 가지 문제점과 궁금증이 생김<br/>
지금부터 질문의 대응 방법과 스파크의 조인 수행 방식에 대해 알아보겠음<br/>
이 내용에서 최적화와 관련된 몇 가지 힌트를 얻을 수 있음<br/>

## 8.11.1 복합 데이터 타입의 조인

불리언을 반환하는 모든 표현식은 조인 표현식으로 간주할 수 있음<br/>

In [25]:
import org.apache.spark.sql.functions.expr

person.withColumnRenamed("id", "personId")
    .join(sparkStatus, expr("array_contains(spark_status, id)")).show(false)

+--------+----------------+----------------+---------------+---+--------------+
|personId|name            |graduate_program|spark_status   |id |status        |
+--------+----------------+----------------+---------------+---+--------------+
|0       |Bill Chambers   |0               |[100]          |100|Contributor   |
|1       |Matei Zaharia   |1               |[500, 250, 100]|500|Vice President|
|1       |Matei Zaharia   |1               |[500, 250, 100]|250|PMC Member    |
|1       |Matei Zaharia   |1               |[500, 250, 100]|100|Contributor   |
|2       |Michael Armbrust|1               |[250, 100]     |250|PMC Member    |
|2       |Michael Armbrust|1               |[250, 100]     |100|Contributor   |
+--------+----------------+----------------+---------------+---+--------------+



import org.apache.spark.sql.functions.expr


## 8.11.2 중복 컬럼명 처리

조인을 수행할 때 가장 까다로운 것 중 하나는 결과 DataFrame에서 중복된 컬럼명을 다루는 것임<br/>
DataFrame의 각 컬럼은 스파크 SQL 엔진인 카탈리스트 내에 고유 ID가 있음<br/>
고유 ID는 카탈리스트 내부에서만 사용할 수 있으며 직접 참조할 수 있는 값은 아님<br/>
그러므로 중복된 컬럼명이 존재하는 DataFrame을 사용할 때는 특정 컬럼을 참조하기 매우 어려움<br/>

이런 문제를 일으키는 두 가지 상황은 다음과 같음<br/>
* 조인에 사용할 DataFrame의 특정 키가 동일한 이름을 가지며, 키가 제거되지 않도록 조인 표현식에 명시하는 경우
* 조인 대상이 아닌 두 개의 컬럼이 동일한 이름을 가진 경우

이러한 상황을 설명하기 위해 잘못된 데이터셋을 만들어보겠음<br/>

In [26]:
val gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")

val joinExpr = gradProgramDupe.col("graduate_program") === person.col("graduate_program")

gradProgramDupe: org.apache.spark.sql.DataFrame = [graduate_program: int, degree: string ... 2 more fields]
joinExpr: org.apache.spark.sql.Column = (graduate_program = graduate_program)


graduate_program 컬럼을 키로 해서 조인을 수행했음에도 불구하고 두 개의 graduate_program 컬럼이 존재함<br/>

In [27]:
person.join(gradProgramDupe, joinExpr).show(false)

+---+----------------+----------------+---------------+----------------+-------+---------------------+-----------+
|id |name            |graduate_program|spark_status   |graduate_program|degree |department           |school     |
+---+----------------+----------------+---------------+----------------+-------+---------------------+-----------+
|0  |Bill Chambers   |0               |[100]          |0               |Masters|School of Information|UC Berkeley|
|2  |Michael Armbrust|1               |[250, 100]     |1               |Ph.D   |EECS                 |UC Berkeley|
|1  |Matei Zaharia   |1               |[500, 250, 100]|1               |Ph.D   |EECS                 |UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+---------------------+-----------+



이러한 컬럼 중 하나를 참조할 때 문제가 발생함

In [28]:
person.join(gradProgramDupe, joinExpr).select("graduate_program").show(false)

org.apache.spark.sql.AnalysisException:  Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.

### 해결 방법 1: 다른 조인 표현식 사용

동일한 이름을 가진 두 개의 키를 사용한다면 가장 쉬운 조치 방법 중 하나는 불리언 형태의 조인 표현식을 문자열이나 시퀀스 형태로 바꾸는 것임<br/>
이렇게 하면 조인을 할 때 두 컬럼 중 하나가 자동으로 제거됨<br/>

In [29]:
person.join(gradProgramDupe, "graduate_program").select("graduate_program").show(false)

+----------------+
|graduate_program|
+----------------+
|0               |
|1               |
|1               |
+----------------+



### 해결 방법 2: 조인 후 컬럼 제거

조인 후에 문제가 되는 컬럼을 제거하는 방법도 있음<br/>
이 경우에는 원본 DataFrame을 사용해 컬럼을 참조해야 함<br/>
조인 시 동일한 키 이름을 사용하거나 원본 DataFrame에 동일한 컬럼명이 존재하는 경우에 사용할 수 있음<br/>

In [30]:
person.join(gradProgramDupe, joinExpr).drop(person.col("graduate_program"))
    .select("graduate_program").show(false)

+----------------+
|graduate_program|
+----------------+
|0               |
|1               |
|1               |
+----------------+



In [31]:
// 이렇게 해도 됨
person.join(gradProgramDupe, joinExpr).drop(gradProgramDupe.col("graduate_program"))
    .select("graduate_program").show(false)

+----------------+
|graduate_program|
+----------------+
|0               |
|1               |
|1               |
+----------------+



In [32]:
val joinExpr = person.col("graduate_program") === graduateProgram.col("id")
person.join(graduateProgram, joinExpr).drop(graduateProgram.col("id")).show(false)

+---+----------------+----------------+---------------+-------+---------------------+-----------+
|id |name            |graduate_program|spark_status   |degree |department           |school     |
+---+----------------+----------------+---------------+-------+---------------------+-----------+
|0  |Bill Chambers   |0               |[100]          |Masters|School of Information|UC Berkeley|
|2  |Michael Armbrust|1               |[250, 100]     |Ph.D   |EECS                 |UC Berkeley|
|1  |Matei Zaharia   |1               |[500, 250, 100]|Ph.D   |EECS                 |UC Berkeley|
+---+----------------+----------------+---------------+-------+---------------------+-----------+



joinExpr: org.apache.spark.sql.Column = (graduate_program = id)


이 방법은 스파크의 SQL 분석 프로세스의 특성을 활용함<br/>
스파크는 명시적으로 참조된 컬럼을 검증할 필요가 없으므로 스파크 코드 분석 단계를 통과함<br/>
위 예제에서 column 함수 대신 col 메서드를 사용한 부분을 주목할 필요가 있음<br/>
col 메서드를 사용함으로써 컬럼 고유의 ID로 해당 컬럼을 암시적으로 지정할 수 있음<br/>

### 해결 방법 3: 조인 전 컬럼명 변경

조인 전에 컬럼명을 변경하면 이런 문제를 완전히 회피할 수 있음

In [33]:
val gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
val joinExpr = person.col("graduate_program") === gradProgram3.col("grad_id")
person.join(gradProgram3, joinExpr).show(false)

+---+----------------+----------------+---------------+-------+-------+---------------------+-----------+
|id |name            |graduate_program|spark_status   |grad_id|degree |department           |school     |
+---+----------------+----------------+---------------+-------+-------+---------------------+-----------+
|0  |Bill Chambers   |0               |[100]          |0      |Masters|School of Information|UC Berkeley|
|2  |Michael Armbrust|1               |[250, 100]     |1      |Ph.D   |EECS                 |UC Berkeley|
|1  |Matei Zaharia   |1               |[500, 250, 100]|1      |Ph.D   |EECS                 |UC Berkeley|
+---+----------------+----------------+---------------+-------+-------+---------------------+-----------+



gradProgram3: org.apache.spark.sql.DataFrame = [grad_id: int, degree: string ... 2 more fields]
joinExpr: org.apache.spark.sql.Column = (graduate_program = grad_id)


# 8.12 스파크의 조인 수행 방식

스파크가 조인을 수행하는 방식을 이해하기 위해서는 실행에 필요한 두 가지 핵심 전략을 이해해야 함<br/>
* 노드 간 네트워크 통신 전략
* 노드별 연산 전략 

이런 내부 동작은 해결하고자 하는 비즈니스 문제와는 관련이 없을 수도 있음<br/>
하지만 스파크 조인 수행 방식을 이해하면 빠르게 완료되는 작업과 절대 완료되지 않는 작업 간의 차이를 알 수 있음<br/>

## 8.12.1 네트워크 통신 전략

스파크는 조인 시 두 가지 클러스터 통신 방식을 활용함<br/>
전체 노드 간 통신을 유발하는 셔플 조인(shuffle join)과 그렇지 않은 브로드캐스트 조인(broadcast join)임<br/>
지금은 두 방식의 세부적인 설명은 생략함<br/>
이런 내부 최적화 기술은 시간이 흘러 비용 기반 옵티마이저(cost-based optimizer, CBO)가 개선되고 더 나은 통신 전략이 도입되는 경우 바뀔 수 있음<br/>
따라서 일반적인 상황에서 정확히 어떤 일이 일어나는지 이해할 수 있도록 고수준 예제를 알아보겠음<br/>
그러면 워크로드의 성능을 빠르고 쉽게 최적화하는 방법을 알 수 있음<br/>

이제부터는 사용자가 스파크에서 사용하는 테이블의 크기가 아주 크거나 아주 작다고 가정함<br/>
물론 실전에서 다루는 테이블의 크기는 다양하므로 중간 크기의 테이블을 활용하는 상황에서 설명과 다르게 동작할 수 있음<br/>
하지만 이해를 돕기 위해 동전의 앞뒷면처럼 단순하게 정의하겠음<br/>

### 큰 테이블과 큰 테이블 조인

하나의 큰 테이블을 다른 큰 테이블과 조인하면 셔플 조인이 발생함<br/>

셔플 조인은 전체 노드 간 통신이 발생함<br/>
그리고 조인에 사용한 특정 키나 키 집합을 어떤 노드가 가졌는지에 따라 해당 노드와 데이터를 공유함<br/>
이런 통신 방식 때문에 네트워크는 복잡해지고 많은 자원을 사용함<br/>
특히 데이터가 잘 나뉘어 있지 않다면 더 심해짐<br/>

셔플 조인 과정은 큰 테이블의 데이터를 다른 큰 테이블의 데이터와 조인하는 과정을 잘 나타냄<br/>
예를 들어 사물인터넷 환경에서 매일 수십억 개의 메시지를 수신하고 일별 변경사항을 식별해야 한다면 deviceId, messageType 그리고 data - 1을 하타내는 컬럼을 이용해 조인할 수 있음<br/>

### 큰 테이블과 작은 테이블 조인

테이블이 단일 워커 노드의 메모리 크기에 적합할 정도(메모리 여유 공간 포함)로 충분히 작은 경우 조인 연산을 최적화할 수 있음<br/>
큰 테이블 사이의 조인에 사용한 방법도 유용하지만 브로드캐스트 조인이 훨씬 효율적임<br/>
이 방법은 작은 DataFrame을 클러스터의 전체 워커 노드에 복제하는 것을 의미함<br/>
이렇게 하면 자원을 많이 사용할 것처럼 보임<br/>
하지만 조인 프로세스 내내 전체 노드가 통신하는 현상을 방지할 수 있음<br/>

*브로드캐스트 조인은 이전 조인 방식과 마찬가지로 대규모 노드 간 통신이 발생함<br/>
하지만 그 이후로는 노드 사이에 추가적인 통신이 발생하지 않음<br/>
따라서 모든 단일 노드에서 개별적으로 조인이 수행되므로 CPU가 가장 큰 병목 구간이 됨*<br/>
다음 예제와 같이 실행 계획을 살펴보면 스파크가 자동으로 데이터셋을 브로드캐스트 조인으로 설정한 것을 알 수 있음<br/>

In [34]:
val joinExpr = person.col("graduate_program") === graduateProgram.col("id")

person.join(graduateProgram, joinExpr).explain()

== Physical Plan ==
*(1) BroadcastHashJoin [graduate_program#15], [id#34], Inner, BuildLeft, false
:- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[2, int, false] as bigint)),false), [id=#686]
:  +- LocalTableScan [id#13, name#14, graduate_program#15, spark_status#16]
+- *(1) LocalTableScan [id#34, degree#35, department#36, school#37]




joinExpr: org.apache.spark.sql.Column = (graduate_program = id)


DataFrame API를 사용하면 옵티마이저에서 브로드캐스트 조인을 사용할 수 있도록 힌트를 줄 수 있음<br/>
힌트를 주는 방법은 broadcast 함수에 작은 크기의 DataFrame을 인수로 전달하는 것임<br/>
다음 예제는 우리가 앞서 보았던 예제와 동일한 실행 계획을 세움<br/>
하지만 항상 동일한 실행 계획을 세우는 것은 아님<br/>

In [35]:
import org.apache.spark.sql.functions.broadcast

val joinExpr = person.col("graduate_program") === graduateProgram.col("id")
person.join(broadcast(graduateProgram), joinExpr).explain()

== Physical Plan ==
*(1) BroadcastHashJoin [graduate_program#15], [id#34], Inner, BuildRight, false
:- *(1) LocalTableScan [id#13, name#14, graduate_program#15, spark_status#16]
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#702]
   +- LocalTableScan [id#34, degree#35, department#36, school#37]




import org.apache.spark.sql.functions.broadcast
joinExpr: org.apache.spark.sql.Column = (graduate_program = id)


물론 단점도 있음<br/>
너무 큰 데이터를 브로드캐스트하면 고비용의 수집 연산이 발생하므로 드라이버 노드가 비정상적으로 종료될 수 있음<br/>
이러한 현상은 향후 개선되어야 하는 영역임<br/>

### 아주 작은 테이블 사이의 조인

아주 작은 테이블 사이의 조인을 할 때는 스파크가 조인 방식을 결정하도록 내버려두는 것이 제일 좋음<br/>
필요한 경우 브로드캐스트 조인을 강제로 지정할 수 있음<br/>

# 8.13 정리

이 장에서는 가장 흔한 사용 사례 중 하나인 조인에 대해 알아보았음<br/>
언급하지는 않았지만, 한 가지 중요하게 고려해야 하는 사항이 있음<br/>
**조인 전에 데이터를 적절하게 분할하면 셔플이 계획되어 있더라도 동일한 머신에 두 DataFrame의 데이터가 있을 수 있음<br/>
따라서 셔플을 피할 수 있고 훨씬 더 효율적으로 실행할 수 있음**<br/>
일부 데이터를 실험용으로 사전에 분할해 조인 수행 시 성능이 향상되는지 확인해볼 것<br/>

다음 장에서는 스파크의 데이터소스 API에 대해 알아보겠음<br/>
데이터소스는 조인 순서를 결정하는 데 부가적인 영향을 미칠 수 있음<br/>
일부 조인은 필터 임무를 수행하므로 네트워크의 교환 데이터를 줄여 워크로드의 성능을 쉽게 향상시킬 수 있음<br/>

다음 장에서는 최근 몇 장에서 사용한 데이터 처리 방식에서 벗어나 구조적 API를 사용해 데이터를 읽고 쓰는 방법에 대해 자세히 알아보겠음<br/>